In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [4]:
!pip install selenium

    100% |████████████████████████████████| 911kB 5.1MB/s ta 0:00:011


In [26]:
def get_lipstick_reviews(link):
    lipstick_reviews = dict()
    options = webdriver.ChromeOptions()
    prefs = {
        'profile.default_content_setting_values' : {
            'images' : 2
        }
    }
    options.add_experimental_option('prefs',prefs)
    driver = webdriver.Chrome(options = options)
    driver.get(link)
    driver.find_element_by_class_name("css-1n39fc0 ").click()
    js="var q=document.documentElement.scrollTop=1000" 
    driver.execute_script(js)
    time.sleep(1)
    try:
        rating = float(driver.find_element_by_class_name("css-ffj77u ").text.split('/')[0].strip())
    except:
        rating = None
    try:
        reco_percent = int(driver.find_element_by_class_name("css-vrylf1 ").text.replace('%','').strip())
    except:
        reco_percent = None
    reviews = list()
    while True:
        try:
            driver.find_element_by_class_name("css-2wz498 ").click()
        except:
            break
    while True:
        try:
            if driver.find_element_by_xpath("//div[@data-comp = 'Review']"):
                for elem in driver.find_elements(By.XPATH, "//div[@data-comp = 'Review']"):
                    try:
                        user_info = elem.find_element_by_class_name("css-1pgaz2s ").text.strip()
                    except:
                        user_info = None
                    user_rating = int(elem.find_element_by_class_name("css-17ol29l ").get_attribute('style').split(':')[1].replace('%;','').strip()) / 20
                    try:
                        user_color = elem.find_element_by_class_name("css-7b8onn ").text.split(':')[1].strip()
                    except:
                        user_color = None
                    try:
                        elem.find_element_by_class_name("css-10o012f ").click()
                        user_review = elem.find_element_by_class_name("css-eq4i08 ").text.strip()
                    except:
                        try:
                            user_review = elem.find_element_by_class_name("css-eq4i08 ").text.strip()
                        except:
                            user_review = None
                    reviews.append({'user_info': user_info,
                                    'user_rating': user_rating,
                                    'user_color': user_color,
                                    'user_review': user_review})
                break
            else:
                break
        except:
            break
    driver.close()
    lipstick_reviews['rating'] = rating
    lipstick_reviews['reco_percent'] = reco_percent
    lipstick_reviews['reviews'] = reviews
    return lipstick_reviews

In [5]:
output = get_lipstick_reviews('https://www.sephora.com/product/la-baton-rouge-lipstick-P425734?icid2=products%20grid:p425734:product')

In [104]:
output

{'rating': 2.8,
 'reco_percent': None,
 'reviews': [{'user_info': 'Eye Color Hazel\nHair color Brunette\nSkin Tone Fair\nSkin Type Dry',
   'user_rating': 3.0,
   'user_color': 'Galocher',
   'user_review': 'Very pretty red, great idea to have both products in one want but I found this formula to be very drying to I returned it.'},
  {'user_info': 'Age 25-34\nEye Color Hazel\nHair color Black\nSkin Tone Light\nSkin Type Combination',
   'user_rating': 2.0,
   'user_color': 'Margaux',
   'user_review': "It's a really neat idea for a product and I wanted to love it, but it was very patchy and drying. I returned it."},
  {'user_info': 'Eye Color Green\nHair color Black\nSkin Tone Light\nSkin Type Dry',
   'user_rating': 5.0,
   'user_color': 'Margaux',
   'user_review': 'What a wonderful lipstick and concept this is! The burgundy color is truly saturated yet applies so easily and lets your own lip show beautifully vs just sitting on it. It melts into the lip and the powder just enhances i

In [103]:
output['reviews'][0]['user_info'].strip().split('\n')

['Eye Color Hazel', 'Hair color Brunette', 'Skin Tone Fair', 'Skin Type Dry']

In [15]:
def search_link(brand_name, product_name, color = ''):
    base_url = 'https://www.sephora.com/search?keyword='
    search_url = brand_name + ' ' + product_name + ' ' + color
    url = base_url + search_url
    return url

In [16]:
product_name = 'Rouge Pur Couture The Slim Matte Lipstick'
brand_name = 'Yves Saint Laurent'
color = '1 Rouge Extravagant bright red'
link = search_link(brand_name, product_name, color)
link

'https://www.sephora.com/search?keyword=Yves Saint Laurent Rouge Pur Couture The Slim Matte Lipstick 1 Rouge Extravagant bright red'

In [17]:
# outcome = get_lipstick_reviews(link)

In [27]:
def get_text(): 
    def search_link(brand_name, product_name, color = ''):
        base_url = 'https://www.sephora.com/search?keyword='
        search_url = brand_name + ' ' + product_name + ' ' + color
        url = base_url + search_url
        return url
    link = search_link(brand_name, product_name, color)
    
    output = get_lipstick_reviews(link)
    text = ''
    listdic_= output['reviews']
    for i in range(len(listdic_)):
        str_= listdic_[i]['user_review']
        text += ''.join(str_)
    return text
# text = get_text()

In [28]:
text = get_text()

In [29]:
text

"The diamond shape definitely threw me for a curve while applying. I would say there is a small learning curve to using it. The color was very rich and velvety that didnt leave my lips sticky. Theres was quite a bit of transfer, over the course of the day I had to apply multiple times. For this price range I expect alot more but for the comfortable texture and feeling it's a fair trade. Most of my longer lasting lipsticks dont feel as comfortable.I received this product for free or complimentary for testing purposes.I’m obsessed with these lipsticks! The formula was like nothing I’ve tried before, such a great matte. The shape is tricky but practice definitely helps.Wow! gorgeous, doesn't bleed, doesn't dry my lips - I'm in love!Beautiful color, awkward shape, messy and dry application. The formula is average at best. Expected from YSL.I liked the pigmentation of this lipstick! The colors are soooo beautiful and bright I love that! I just don’t like about it that it do transfer and tha

In [30]:
def get_pos_neg_words():
    def get_words(url):
        import requests
        words = requests.get(url).content.decode('latin-1')
        word_list = words.split('\n')
        index = 0
        while index < len(word_list):
            word = word_list[index]
            if ';' in word or not word:
                word_list.pop(index) #pop删掉的是第几位的数字 pop(index)
            else:
                index+=1
        return word_list

    #Get lists of positive and negative words
    p_url = 'http://ptrckprry.com/course/ssd/data/positive-words.txt'
    n_url = 'http://ptrckprry.com/course/ssd/data/negative-words.txt'
    positive_words = get_words(p_url)
    negative_words = get_words(n_url)
    return positive_words,negative_words

positive_words,negative_words = get_pos_neg_words()

In [31]:
def do_pos_neg_sentiment_analysis(text,debug=False):
    positive_words,negative_words = get_pos_neg_words()
    from nltk import word_tokenize
    results = list()
    cpos = cneg = lpos = lneg = 0
    for word in word_tokenize(text):
        if word in positive_words:
            if debug:
                print("Positive",word)
            cpos+=1
        if word in negative_words:
            if debug:
                print("Negative",word)
            cneg+=1
    results.append((product_name, cpos/len(word_tokenize(text)),cneg/len(word_tokenize(text))))
    return results

In [32]:
# text= get_text()
# do_pos_neg_sentiment_analysis(text,debug=False)


[('Rouge Pur Couture The Slim Matte Lipstick',
  0.06615093148089675,
  0.028102305020524156)]

In [33]:
# Text- Summerization
def build_summary(text):
    from nltk.tokenize import word_tokenize
    from nltk.tokenize import sent_tokenize
    from nltk.probability import FreqDist
    from nltk.corpus import stopwords
    from collections import OrderedDict
    summary_sentences = []
    candidate_sentences = {}
    candidate_sentence_counts = {}
    
    words = word_tokenize(text)
    
    lowercase_words = [word.lower() for word in words
                      if word not in stopwords.words() and word.isalpha()]
    
    word_frequencies = FreqDist(lowercase_words)
    most_frequent_words = FreqDist(lowercase_words).most_common(20)
    sentences = sent_tokenize(text)
    for sentence in sentences:
        candidate_sentences[sentence] = sentence.lower()
    for long, short in candidate_sentences.items():
        count = 0
        for freq_word, frequency_score in most_frequent_words:
            if freq_word in short:
                count += frequency_score
                candidate_sentence_counts[long] = count   
    sorted_sentences = OrderedDict(sorted(
                        candidate_sentence_counts.items(),
                        key = lambda x: x[1],
                        reverse = True)[:2])
    return sorted_sentences   

In [34]:
# build_naive_summary(text)
summary = '\n'.join(build_summary(text))
print(summary)

Had to buy from YSL because color was sold out from SephoraLiterally Great , goes on greatamazing and I love it but I have not tried it yetI loved the color and the nifty pointed tip for lining but the formula wasn’t as long lasting as I had hoped.not worth the price, kind of patchy, but like the packaging and shape.I think that this lipstick is definitely long lasting so if that’s what you’re looking for this will be great.
I would definitely recommend it.I love this lipstick, the color is amazing, I love the shape and slimness of it.
